In [1]:
import pandas as pd

df = pd.read_stata("./serrano/serrano_2024_Stata/serrano.dta")


['ORGNR', 'KOMORGNR', 'KOMNAMN', 'BSTYP', 'STATUS', 'REDTYP', 'BELKOD', 'UTDBEL', 'NTOMS', 'LAGERF', 'AKTARB', 'ROINTOV1', 'RAVAR', 'HANDVAR', 'EXTKOSOV', 'PERSKOS', 'AVSKRIV', 'JFRST1', 'RORKOOV1', 'RORRESUL', 'RESAND', 'RTEINKNC', 'RTEINEXT', 'RTEINOV', 'RTEKOKNC', 'RTEKOEXT', 'RTEKOOV', 'JFRSTFIN', 'RESEFIN', 'EXTRAINT', 'EXTRAKOS', 'KNCBDR', 'AGTSK', 'BSLDISP', 'SKATTER', 'MININTRR', 'RESAR', 'KOSALVAR', 'BRUTORES', 'FORSKO', 'ADMKO', 'FOUKO', 'JFRST2', 'ROINTOV2', 'RORKOOV2', 'EJINBET', 'FOUBAUTG', 'PATLIC', 'GOODWILL', 'IMANLOV', 'IMANLSU', 'BYGGMARK', 'MASK', 'INVENT', 'MASKINV', 'MATANLOV', 'MATANLSU', 'ANDKNC', 'LFORDKNC', 'LANDELAG', 'FIANLTOV', 'FIANLTSU', 'ANLTSU', 'PAGARB', 'LAGEROV', 'LAGERSU', 'KUNDFORD', 'KFORDKNC', 'KFORDOV', 'KFORDSU', 'KPLACSU', 'KABASU', 'OMSTGSU', 'TILLGSU', 'AKTIEKAP', 'OVERKURS', 'UPPSKR', 'OVRGBKAP', 'BALRES', 'KNCBDREL', 'AGTSKEL', 'RESARB', 'EKSU', 'OBESKRES', 'MININTR', 'AVSSU', 'LSKKRIN', 'LSKKNC', 'LSKOV', 'LSKSU', 'KSKKRIN', 'KSKLEV', 'KSKKNC', 'KSKOV', 'KSKSU', 'EKSKSU', 'RTENTO', 'ANTANST', 'LONLEDN', 'LONOV', 'SOCKOSTN', 'TANTLEDN', 'RESLONOV', 'AVGVED', 'AVSKSALV', 'AVSKFSG', 'AVSKADM', 'AVSKFOU', 'AVSKOV2', 'AVSKOSPC', 'INTFTG', 'INTFAST', 'SAKOV', 'SAKKOM', 'SAKSU', 'AGTSKV', 'ANSVFOV', 'ANSVFKOM', 'ANSVFSU', 'CHKRBEV', 'CHKRUTN', 'REVBER', 'BOLSTPRO', 'MODDTM', 'BSLSTART', 'BSLSLUT']

In [2]:
# Filter the DataFrame
filtered_df = df[(df['ser_year'] >= 2007) & # remove data before 2007
                 (df['ser_jurform'] == 49) & # aktiebolag
                 (df['ser_aktiv'] == 1) & # active companies
                 (df['ser_ftgkategori'] == 30)] # private companies, i.e., not state-owned etc.



# Calculate the annual growth rate for each company and add it as a new column
filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()

# Identify high-growth periods and create a new binary column
filtered_df['HIGH_GROWTH'] = filtered_df.groupby('ORGNR').apply(
    lambda x: ((x['TURNOVER_GROWTH'] > 0.20).rolling(window=3).sum() == 3) & 
              (x['ser_stklf'].iloc[0] in [3, 4, 5, 6, 7]) # only consider companies with at least 10 employees, based on size category
).reset_index(level=0, drop=True).astype(int)



/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/1801518813.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()
/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/1801518813.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()
/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/1801518813.py:10: SettingWithCopyWarning: 
A value is tryi

In [5]:
# Group by 'ORGNR' and count the number of rows where 'ser_stklf' is 0
zero_employee_counts = filtered_df[filtered_df['ser_stklf'] == 0].groupby('ORGNR').size()

# Identify companies with more than 4 rows where 'ser_stklf' is 0
companies_to_remove = zero_employee_counts[zero_employee_counts > 4].index

# Filter out rows associated with these companies
filtered_df = filtered_df[~filtered_df['ORGNR'].isin(companies_to_remove)]

In [7]:
filtered_df['ORGNR'].nunique() # amount of companies in this filtered dataset.

529430

In [8]:
# add one industry to all companies.
dict_of_industries = {
    10: 'Energy & Environment',
    15: 'Materials',
    20: 'Industrial goods',
    22: 'Construction industry',
    25: 'Shopping goods',
    30: 'Convenience goods',
    35: 'Health & Education',
    40: 'Finance & Real estate',
    45: 'IT & Electronics',
    50: 'Telecom & Media',
    60: 'Corporate services',
    98: 'Other',
    99: 'SNI07 missing'
}

# Determine the most frequent 'bransch_borsbransch_konv' value for each company
most_frequent_industry = filtered_df.groupby('ORGNR')['bransch_borsbransch_konv'].agg(lambda x: x.mode()[0])

# Map the most frequent 'bransch_borsbransch_konv' value to the corresponding industry name
most_frequent_industry = most_frequent_industry.map(dict_of_industries)

# Add the new 'INDUSTRY' column to the DataFrame
filtered_df = filtered_df.merge(most_frequent_industry.rename('INDUSTRY'), on='ORGNR')

In [9]:
# Filter out rows where the 'INDUSTRY' column is 'SNI07 missing' or 'Other'
filtered_df = filtered_df[~filtered_df['INDUSTRY'].isin(['SNI07 missing', 'Other'])]

In [10]:
print(len(filtered_df))
print(len(filtered_df['ORGNR'].unique()))

3742401
488611


# Add varibles from Vanacker & Manigart (2010)

In [13]:
# Internal Finance
# N.D., this is only retained earnings, not profit/loss of the current year, maybe that should be included too.
# Maybe not since that profit might be used for dividends so then maybe we list it as a internal finance thing but in reailty that money is not used for investments.

# Calculate the percentage change in retained earnings from the previous year
filtered_df['br10e_balres_pct_change'] = filtered_df.groupby('ORGNR')['br10e_balres'].pct_change()

# Calculate the percentage increase in retained earnings relative to total assets
filtered_df['INTERNAL_FINANCE'] = (filtered_df['br10e_balres_pct_change'] * filtered_df.groupby('ORGNR')['br10e_balres'].shift(1) / filtered_df['br09_tillgsu']) > 0.05

# Convert the boolean values to binary (1 or 0)
filtered_df['INTERNAL_FINANCE'] = filtered_df['INTERNAL_FINANCE'].astype(int)

# Drop the temporary column
filtered_df = filtered_df.drop(columns=['br10e_balres_pct_change'])

/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/2737011162.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_df['br10e_balres_pct_change'] = filtered_df.groupby('ORGNR')['br10e_balres'].pct_change()


Filtered data with internal finance saved to ./serrano/serrano_2024_Stata/filtered_firms_with_internal_finance.xlsx


In [ ]:
# Financial Debt

# Calculate the combined financial debt
filtered_df['combined_financial_debt'] = filtered_df['br14_kskkrin'] + filtered_df['br16_lskkrin']

# Calculate the percentage change in combined financial debt from the previous year
filtered_df['combined_financial_debt_pct_change'] = filtered_df.groupby('ORGNR')['combined_financial_debt'].pct_change()

# Calculate the percentage increase in combined financial debt relative to total assets
filtered_df['FINANCIAL_DEBT'] = (filtered_df['combined_financial_debt_pct_change'] * filtered_df.groupby('ORGNR')['combined_financial_debt'].shift(1) / filtered_df['br09_tillgsu']) > 0.05

# Convert the boolean values to binary (1 or 0)
filtered_df['FINANCIAL_DEBT'] = filtered_df['FINANCIAL_DEBT'].astype(int)

# Drop the temporary columns
filtered_df = filtered_df.drop(columns=['combined_financial_debt', 'combined_financial_debt_pct_change'])

/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/499950134.py:7: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_df['combined_financial_debt_pct_change'] = filtered_df.groupby('ORGNR')['combined_financial_debt'].pct_change()


Filtered data with internal finance and financial debt saved to ./serrano/serrano_2024_Stata/filtered_firms_with_internal_finance_and_financial_debt.xlsx


In [16]:
# External Equity

# Calculate the combined external equity
filtered_df['external_equity'] = filtered_df['br10a_aktiekap'] + filtered_df['br10b_overkurs']

# Calculate the percentage change in external equity from the previous year
filtered_df['external_equity_pct_change'] = filtered_df.groupby('ORGNR')['external_equity'].pct_change()

# Calculate the percentage increase in external equity relative to total assets
filtered_df['EXTERNAL_EQUITY'] = (filtered_df['external_equity_pct_change'] * filtered_df.groupby('ORGNR')['external_equity'].shift(1) / filtered_df['br09_tillgsu']) > 0.05

# Convert the boolean values to binary (1 or 0)
filtered_df['EXTERNAL_EQUITY'] = filtered_df['EXTERNAL_EQUITY'].astype(int)

# Drop the temporary columns
filtered_df = filtered_df.drop(columns=['external_equity', 'external_equity_pct_change'])

/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/3300434013.py:7: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_df['external_equity_pct_change'] = filtered_df.groupby('ORGNR')['external_equity'].pct_change()


Filtered data with internal finance, financial debt, and external equity saved to ./serrano/serrano_2024_Stata/filtered_firms_with_internal_finance_financial_debt_and_external_equity.xlsx


In [26]:
# Group by 'ser_year' and calculate the percentage of firms with 1 in each column
summary_table = filtered_df.groupby('ser_year').apply(lambda x: pd.Series({
    'Internal finance %': (x['INTERNAL_FINANCE'].sum() / len(x)) * 100,
    'Financial debt %': (x['FINANCIAL_DEBT'].sum() / len(x)) * 100,
    'External equity %': (x['EXTERNAL_EQUITY'].sum() / len(x)) * 100
}))

# Display the summary table
print(summary_table)

          Internal finance %  Financial debt %  External equity %
ser_year                                                         
2007.0              0.000000          0.000000           0.000000
2008.0             23.747299          8.270698           0.546476
2009.0             21.048502          7.009052           0.561924
2010.0             20.527704          6.678210           0.480981
2011.0             22.425112          6.636156           0.207089
2012.0             21.910987          6.253410           0.183005
2013.0             20.447692          5.660736           0.169523
2014.0             23.330864          5.391598           0.171357
2015.0             24.501878          5.199335           0.180466
2016.0             24.259159          5.284724           0.163076
2017.0             23.330448          5.287637           0.121587
2018.0             26.690854          5.170140           0.117169
2019.0             27.124858          4.762405           0.097000
2020.0    

/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/3889269946.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_table = filtered_df.groupby('ser_year').apply(lambda x: pd.Series({


In [25]:
# Create DataFrame with high-growth companies
high_growth_orgnr = filtered_df[filtered_df['HIGH_GROWTH'] == 1]['ORGNR'].unique()
high_growth_df = filtered_df[filtered_df['ORGNR'].isin(high_growth_orgnr)]

# Create DataFrame with non-high-growth companies
non_high_growth_df = filtered_df[~filtered_df['ORGNR'].isin(high_growth_orgnr)]

# Create a DataFrame with unique high-growth companies
unique_high_growth_df = high_growth_df.drop_duplicates(subset='ORGNR')

# Create a DataFrame with unique non-high-growth companies
unique_non_high_growth_df = non_high_growth_df.drop_duplicates(subset='ORGNR')
print(len(unique_non_high_growth_df))



485744


In [19]:
# Combine the unique high-growth companies with the non-high-growth companies
combined_df = pd.concat([unique_high_growth_df, unique_non_high_growth_df])

print(len(combined_df))
combined_df['ORGNR'].nunique()


488611


488611

In [27]:
# Calculate the percentage of each industry among all companies
industry_percentage = combined_df['INDUSTRY'].value_counts(normalize=True) * 100

# Calculate the total percentage to check for rounding errors
total_percentage = industry_percentage.sum()

# If the total percentage is not exactly 100, adjust the last value
if total_percentage != 100:
    difference = 100 - total_percentage
    industry_percentage.iloc[-1] += difference

# Add a row for the total percentage
industry_percentage['Total'] = industry_percentage.sum()

# Display the result
print(industry_percentage)

# Calculate the percentage of each industry among high-growth companies
industry_percentage = unique_high_growth_df['INDUSTRY'].value_counts(normalize=True) * 100

# Calculate the total percentage to check for rounding errors
total_percentage = industry_percentage.sum()

# If the total percentage is not exactly 100, adjust the last value
if total_percentage != 100:
    difference = 100 - total_percentage
    industry_percentage.iloc[-1] += difference

# Add a row for the total percentage
industry_percentage['Total'] = industry_percentage.sum()

# Display the result
print(industry_percentage)

INDUSTRY
Corporate services        30.203782
Shopping goods            21.698857
Construction industry     14.701675
Health & Education         6.917978
IT & Electronics           6.683845
Finance & Real estate      6.570462
Industrial goods           5.921070
Convenience goods          3.622309
Telecom & Media            1.947152
Materials                  1.244753
Energy & Environment       0.488118
Total                    100.000000
Name: proportion, dtype: float64
INDUSTRY
Corporate services        25.776073
Construction industry     19.532612
Shopping goods            14.893617
Health & Education        12.452040
Industrial goods           9.208232
IT & Electronics           8.161842
Convenience goods          3.348448
Finance & Real estate      3.034531
Telecom & Media            1.639344
Materials                  1.185909
Energy & Environment       0.767353
Total                    100.000000
Name: proportion, dtype: float64


In [28]:
# Calculate the average goodwill for each industry among all companies
avg_goodwill_by_industry_all = combined_df.groupby('INDUSTRY')['br01c_goodwill'].mean()

# Sort by average goodwill in descending order
average_goodwill_by_industry = avg_goodwill_by_industry_all.sort_values(ascending=False)

# Display the result
print(average_goodwill_by_industry)

# Calculate the average goodwill for each industry among HGFs
avg_goodwill_by_industry_high_growth = unique_high_growth_df.groupby('INDUSTRY')['br01c_goodwill'].mean()

# Sort by average goodwill in descending order
avg_goodwill_by_industry_high_growth = avg_goodwill_by_industry_high_growth.sort_values(ascending=False)

# Display the result
print(avg_goodwill_by_industry_high_growth)

INDUSTRY
Energy & Environment     775.081444
Materials                719.238926
Industrial goods         504.911862
Telecom & Media          473.896490
Convenience goods        350.011102
Health & Education       154.906440
IT & Electronics         153.526295
Shopping goods           109.110065
Finance & Real estate     71.700881
Corporate services        53.337668
Construction industry     25.057200
Name: br01c_goodwill, dtype: float64
INDUSTRY
Energy & Environment     1574.681818
Industrial goods         1268.776515
IT & Electronics          729.713675
Shopping goods            591.971897
Finance & Real estate     561.574713
Telecom & Media           477.297872
Health & Education        339.000000
Corporate services        178.174560
Convenience goods         134.520833
Construction industry      90.646429
Materials                  25.617647
Name: br01c_goodwill, dtype: float64


In [29]:
# Calculate the average goodwill for each industry among all companies
avg_sales_by_industry_all = combined_df.groupby('INDUSTRY')['rr01_ntoms'].mean()

# Sort by average goodwill in descending order
average_sales_by_industry = avg_goodwill_by_industry_all.sort_values(ascending=False)

# Display the result
print(average_sales_by_industry)

# Calculate the average goodwill for each industry among HGFs
avg_sales_by_industry_high_growth = unique_high_growth_df.groupby('INDUSTRY')['rr01_ntoms'].mean()

# Sort by average goodwill in descending order
avg_sales_by_industry_high_growth = avg_sales_by_industry_high_growth.sort_values(ascending=False)

# Display the result
print(avg_sales_by_industry_high_growth)

INDUSTRY
Energy & Environment     775.081444
Materials                719.238926
Industrial goods         504.911862
Telecom & Media          473.896490
Convenience goods        350.011102
Health & Education       154.906440
IT & Electronics         153.526295
Shopping goods           109.110065
Finance & Real estate     71.700881
Corporate services        53.337668
Construction industry     25.057200
Name: br01c_goodwill, dtype: float64
INDUSTRY
Energy & Environment     612610.318182
Materials                209139.882353
Finance & Real estate     87681.678161
Industrial goods          71223.469697
Convenience goods         47523.437500
Shopping goods            41027.800937
IT & Electronics          31189.606838
Construction industry     30803.689286
Corporate services        25940.364005
Telecom & Media           25127.085106
Health & Education        21918.308123
Name: rr01_ntoms, dtype: float64


In [30]:
# Assuming 'combined_df' is your DataFrame and 'ser_stklf' is the column for size category

# Calculate the average size category for each industry among all companies
avg_size_by_industry_all = combined_df.groupby('INDUSTRY')['ser_stklf'].mean()

# Sort by average size category in descending order
avg_size_by_industry_all = avg_size_by_industry_all.sort_values(ascending=False)

# Display the result
print("Average size category by industry (all companies):")
print(avg_size_by_industry_all)

# Calculate the average size category for each industry among HGFs
avg_size_by_industry_high_growth = unique_high_growth_df.groupby('INDUSTRY')['ser_stklf'].mean()

# Sort by average size category in descending order
avg_size_by_industry_high_growth = avg_size_by_industry_high_growth.sort_values(ascending=False)

# Display the result
print("\nAverage size category by industry (high-growth firms):")
print(avg_size_by_industry_high_growth)

Average size category by industry (all companies):
INDUSTRY
Industrial goods         1.612215
Materials                1.396251
Convenience goods        1.334991
Energy & Environment     1.208386
Construction industry    1.163126
Shopping goods           1.139479
Health & Education       1.092391
Corporate services       1.017381
IT & Electronics         1.010135
Telecom & Media          0.994639
Finance & Real estate    0.644655
Name: ser_stklf, dtype: float64

Average size category by industry (high-growth firms):
INDUSTRY
Materials                3.941176
Energy & Environment     3.909091
Industrial goods         3.625000
Telecom & Media          3.617021
Health & Education       3.610644
Finance & Real estate    3.505747
Shopping goods           3.494145
Corporate services       3.488498
IT & Electronics         3.470085
Convenience goods        3.416667
Construction industry    3.385714
Name: ser_stklf, dtype: float64


In [18]:
# Save the filtered DataFrame to an Excel file
output_path = "./serrano/serrano_2024_Stata/firms.xlsx"
filtered_df.head(20000).to_excel(output_path, index=False)

print(f"Filtered data saved to {output_path}")

Filtered data saved to ./serrano/serrano_2024_Stata/firms.xlsx


In [31]:
# Group by 'ser_year' and calculate the percentage of firms with 1 in each column
summary_table = combined_df.groupby('ser_year').apply(lambda x: pd.Series({
    'Internal finance %': (x['INTERNAL_FINANCE'].sum() / len(x)) * 100,
    'Financial debt %': (x['FINANCIAL_DEBT'].sum() / len(x)) * 100,
    'External equity %': (x['EXTERNAL_EQUITY'].sum() / len(x)) * 100
}))

# Display the summary table
print(summary_table)

          Internal finance %  Financial debt %  External equity %
ser_year                                                         
2007.0                   0.0               0.0                0.0
2008.0                   0.0               0.0                0.0
2009.0                   0.0               0.0                0.0
2010.0                   0.0               0.0                0.0
2011.0                   0.0               0.0                0.0
2012.0                   0.0               0.0                0.0
2013.0                   0.0               0.0                0.0
2014.0                   0.0               0.0                0.0
2015.0                   0.0               0.0                0.0
2016.0                   0.0               0.0                0.0
2017.0                   0.0               0.0                0.0
2018.0                   0.0               0.0                0.0
2019.0                   0.0               0.0                0.0
2020.0    

/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_72693/6120956.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_table = combined_df.groupby('ser_year').apply(lambda x: pd.Series({
